In [1]:
import pandas as pd
import nltk
import operator
from pylab import *
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
calendar = pd.read_csv('calendar.csv')
calendar.head()

,listing_id,date,available,price
0,12147973,2017-09-05,f,NaN
1,12147973,2017-09-04,f,NaN
2,12147973,2017-09-03,f,NaN
3,12147973,2017-09-02,f,NaN
4,12147973,2017-09-01,f,NaN


In [3]:
listings = pd.read_csv('listings.csv')
listings.head()

,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,...,review_scores_value,requires_license,license,jurisdiction_names,instant_bookable,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month
0,12147973,https://www.airbnb.com/rooms/12147973,20160906204935,2016-09-07,Sunny Bungalow in the City,"Cozy, sunny, family home. Master bedroom high...",The house has an open and cozy feel at the sam...,"Cozy, sunny, family home. Master bedroom high...",none,"Roslindale is quiet, convenient and friendly. ...",...,NaN,f,NaN,NaN,f,moderate,f,f,1,NaN
1,3075044,https://www.airbnb.com/rooms/3075044,20160906204935,2016-09-07,Charming room in pet friendly apt,Charming and quiet room in a second floor 1910...,Small but cozy and quite room with a full size...,Charming and quiet room in a second floor 1910...,none,"The room is in Roslindale, a diverse and prima...",...,9.0,f,NaN,NaN,t,moderate,f,f,1,1.30
2,6976,https://www.airbnb.com/rooms/6976,20160906204935,2016-09-07,Mexican Folk Art Haven in Boston,"Come stay with a friendly, middle-aged guy in ...","Come stay with a friendly, middle-aged guy in ...","Come stay with a friendly, middle-aged guy in ...",none,The LOCATION: Roslindale is a safe and diverse...,...,10.0,f,NaN,NaN,f,moderate,t,f,1,0.47
3,1436513,https://www.airbnb.com/rooms/1436513,20160906204935,2016-09-07,Spacious Sunny Bedroom Suite in Historic Home,Come experience the comforts of home away from...,Most places you find in Boston are small howev...,Come experience the comforts of home away from...,none,Roslindale is a lovely little neighborhood loc...,...,10.0,f,NaN,NaN,f,moderate,f,f,1,1.00
4,7651065,https://www.airbnb.com/rooms/7651065,20160906204935,2016-09-07,Come Home to Boston,"My comfy, clean and relaxing home is one block...","Clean, attractive, private room, one block fro...","My comfy, clean and relaxing home is one block...",none,"I love the proximity to downtown, the neighbor...",...,10.0,f,NaN,NaN,f,flexible,f,f,1,2.25


In [4]:
reviews = pd.read_csv('reviews.csv')
reviews.head()

,listing_id,id,date,reviewer_id,reviewer_name,comments
0,1178162,4724140,2013-05-21,4298113,Olivier,My stay at islam's place was really cool! Good...
1,1178162,4869189,2013-05-29,6452964,Charlotte,Great location for both airport and city - gre...
2,1178162,5003196,2013-06-06,6449554,Sebastian,We really enjoyed our stay at Islams house. Fr...
3,1178162,5150351,2013-06-15,2215611,Marine,The room was nice and clean and so were the co...
4,1178162,5171140,2013-06-16,6848427,Andrew,Great location. Just 5 mins walk from the Airp...


In [5]:
print('Calendar shape is ', calendar.shape)
print('Listings shape is ', listings.shape)
print('Reviews shape is', reviews.shape)

Calendar shape is  (1308890, 4)
Listings shape is  (3585, 95)
Reviews shape is (68275, 6)


In [6]:
# calendar['available'].value_counts()
calendar.groupby('listing_id')['available'].value_counts(normalize=True)

listing_id  available
3353        t            0.682192
            f            0.317808
5506        t            0.942466
            f            0.057534
6695        t            0.887671
            f            0.112329
6976        t            0.873973
            f            0.126027
8792        t            0.679452
            f            0.320548
9273        t            0.997260
            f            0.002740
9765        t            0.991781
            f            0.008219
9824        t            0.884932
            f            0.115068
9855        t            0.997260
            f            0.002740
9857        t            0.950685
            f            0.049315
9858        t            0.997260
            f            0.002740
9860        t            0.967123
            f            0.032877
9870        t            0.991781
            f            0.008219
9903        t            0.997260
            f            0.002740
10730       f            0

In [7]:
import re
from nltk.corpus import stopwords
StopWordsList = stopwords.words("english")

# this function is to be mapped over an entire column of messages, posts, reviews, etc. It converts each cell's contents
# into a string, makes it lower case, splits it into words
def string_process(s):
    s=str(s)
    lower = s.lower()
    words = lower.split()
    refined = []
    for i in words:
        if i not in StopWordsList:
            refined.append(re.sub("[^\w]+", "", i))
    sentence = ' '.join([word for word in refined])
    return(sentence)

In [8]:
listings['SUMMARY'] = listings['summary'].map(string_process)
reviews['COMMENTS'] = reviews['comments'].map(string_process)

In [9]:
reviews['COMMENTS'][:10]

0    stay islams place really cool good location 5m...
1    great location airport city  great amenities h...
2    really enjoyed stay islams house outside house...
3    room nice clean commodities close airport metr...
4    great location 5 mins walk airport station goo...
5    truely exeptional place stay hosts amazingly f...
6    really nice time boston  best place stay bosto...
7    islam nice guy  attentive funny gave recommand...
8    place really well furnished pleasant clean isl...
9    stay islams place fantastic couldnt asked bett...
Name: COMMENTS, dtype: object

In [10]:
corpus = reviews['COMMENTS'].sum()

In [11]:
corpus_words = nltk.word_tokenize(corpus.lower())
corpus_words = [word for word in corpus_words if word.isalpha()==True]
fdist = nltk.FreqDist(corpus_words)
sorted_fdist = pd.DataFrame(sorted(fdist.items(), key=operator.itemgetter(1),reverse=True))
sorted_fdist.columns = ['word','frequency']
# top_20 = sorted_fdist[:20]

#look at frequency to define attributes
sorted_fdist[:50]

,word,frequency
0,great,33617
1,stay,31988
2,place,27643
3,boston,27336
4,apartment,25536
5,clean,22357
6,location,21275
7,us,21156
8,room,18748
9,would,18723


In [12]:
# next 50...
sorted_fdist[:150][99:]

,word,frequency
99,thank,3588
100,excellent,3587
101,safe,3580
102,trip,3537
103,cozy,3532
104,access,3531
105,amazing,3519
106,breakfast,3451
107,sure,3410
108,small,3400


In [13]:


from nltk.sentiment.vader import SentimentIntensityAnalyzer
sid = SentimentIntensityAnalyzer()
from nltk.corpus import stopwords   # stopwords to detect language
from nltk import wordpunct_tokenize # function to split up our words

def get_language_likelihood(input_text):
    """Return a dictionary of languages and their likelihood of being the 
    natural language of the input text
    """
 
    input_text = input_text.lower()
    input_words = wordpunct_tokenize(input_text)
 
    language_likelihood = {}
    total_matches = 0
    for language in stopwords._fileids:
        language_likelihood[language] = len(set(input_words) &
                set(stopwords.words(language)))
 
    return language_likelihood
 
def get_language(input_text):
    """Return the most likely language of the given text
    """ 
    likelihoods = get_language_likelihood(input_text)
    return sorted(likelihoods, key=likelihoods.get, reverse=True)[0]

C:\Users\nsche\Anaconda3\lib\site-packages\nltk\twitter\__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [ ]:
reviews_text = reviews['COMMENTS']
reviews_text[:5]

In [15]:
### testing out sentiment score concept

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

# loading the SentimentIntensityAnalyser object and assigning it
# to another variable for ease of use
analyzer = SentimentIntensityAnalyzer()

### polarity_scores() method to get the sentiment metrics for a piece of text
def print_sentiment_scores(sentence):
    snt = analyzer.polarity_scores(sentence)
    print("{:-<40} {}".format(sentence, str(snt)))


In [16]:
print_sentiment_scores("I just got a call from my boss - does he realise it's Saturday? smh :(")

I just got a call from my boss - does he realise it's Saturday? smh :( {'neg': 0.321, 'neu': 0.679, 'pos': 0.0, 'compound': -0.6369}


In [17]:
print_sentiment_scores("The food is really GOOD! But the service is dreadful")

The food is really GOOD! But the service is dreadful {'neg': 0.192, 'neu': 0.529, 'pos': 0.279, 'compound': 0.3222}


In [18]:
comments = reviews['COMMENTS']
comments[:5]

0    stay islams place really cool good location 5m...
1    great location airport city  great amenities h...
2    really enjoyed stay islams house outside house...
3    room nice clean commodities close airport metr...
4    great location 5 mins walk airport station goo...
Name: COMMENTS, dtype: object

In [24]:
### getting sentiment scores
#using analyzer instantiated above
reviews['sentiment'] = reviews['COMMENTS'].map(analyzer.polarity_scores)

reviews['sentiment'][:10]

0    {'neg': 0.0, 'neu': 0.57, 'pos': 0.43, 'compou...
1    {'neg': 0.0, 'neu': 0.5, 'pos': 0.5, 'compound...
2    {'neg': 0.093, 'neu': 0.623, 'pos': 0.284, 'co...
3    {'neg': 0.0, 'neu': 0.5, 'pos': 0.5, 'compound...
4    {'neg': 0.0, 'neu': 0.508, 'pos': 0.492, 'comp...
5    {'neg': 0.0, 'neu': 0.725, 'pos': 0.275, 'comp...
6    {'neg': 0.0, 'neu': 0.471, 'pos': 0.529, 'comp...
7    {'neg': 0.0, 'neu': 0.589, 'pos': 0.411, 'comp...
8    {'neg': 0.064, 'neu': 0.401, 'pos': 0.536, 'co...
9    {'neg': 0.089, 'neu': 0.563, 'pos': 0.348, 'co...
Name: sentiment, dtype: object

In [25]:
reviews[:10]

,listing_id,id,date,reviewer_id,reviewer_name,comments,COMMENTS,sentiment
0,1178162,4724140,2013-05-21,4298113,Olivier,My stay at islam's place was really cool! Good...,stay islams place really cool good location 5m...,"{'neg': 0.0, 'neu': 0.57, 'pos': 0.43, 'compou..."
1,1178162,4869189,2013-05-29,6452964,Charlotte,Great location for both airport and city - gre...,great location airport city great amenities h...,"{'neg': 0.0, 'neu': 0.5, 'pos': 0.5, 'compound..."
2,1178162,5003196,2013-06-06,6449554,Sebastian,We really enjoyed our stay at Islams house. Fr...,really enjoyed stay islams house outside house...,"{'neg': 0.093, 'neu': 0.623, 'pos': 0.284, 'co..."
3,1178162,5150351,2013-06-15,2215611,Marine,The room was nice and clean and so were the co...,room nice clean commodities close airport metr...,"{'neg': 0.0, 'neu': 0.5, 'pos': 0.5, 'compound..."
4,1178162,5171140,2013-06-16,6848427,Andrew,Great location. Just 5 mins walk from the Airp...,great location 5 mins walk airport station goo...,"{'neg': 0.0, 'neu': 0.508, 'pos': 0.492, 'comp..."
5,1178162,5198929,2013-06-17,6663826,Arndt,A truely exeptional place to stay. The hosts a...,truely exeptional place stay hosts amazingly f...,"{'neg': 0.0, 'neu': 0.725, 'pos': 0.275, 'comp..."
6,1178162,6702817,2013-08-21,8099222,Maurice,It was a really nice time in Boston - best pla...,really nice time boston best place stay bosto...,"{'neg': 0.0, 'neu': 0.471, 'pos': 0.529, 'comp..."
7,1178162,6873023,2013-08-28,7671888,Elodie,"Islam is a very nice guy ! Attentive, funny, h...",islam nice guy attentive funny gave recommand...,"{'neg': 0.0, 'neu': 0.589, 'pos': 0.411, 'comp..."
8,1178162,7646702,2013-09-28,8197342,Arkadiusz,"The place is really well furnished, pleasant a...",place really well furnished pleasant clean isl...,"{'neg': 0.064, 'neu': 0.401, 'pos': 0.536, 'co..."
9,1178162,8094418,2013-10-15,9040491,Matthew,Our stay at Islam's place was fantastic. We co...,stay islams place fantastic couldnt asked bett...,"{'neg': 0.089, 'neu': 0.563, 'pos': 0.348, 'co..."


In [27]:
# pulling scores out of dictionary and adding them as comments

reviews['snt_pos'] = reviews['sentiment'].map(lambda v: v['pos'])
reviews['snt_neg'] = reviews['sentiment'].map(lambda v: v['neg'])
reviews['snt_neu'] = reviews['sentiment'].map(lambda v: v['neu'])
reviews['snt_compound'] = reviews['sentiment'].map(lambda v: v['compound'])

reviews[:10]

,listing_id,id,date,reviewer_id,reviewer_name,comments,COMMENTS,sentiment,snt_pos,snt_neg,snt_neu,snt_compound
0,1178162,4724140,2013-05-21,4298113,Olivier,My stay at islam's place was really cool! Good...,stay islams place really cool good location 5m...,"{'neg': 0.0, 'neu': 0.57, 'pos': 0.43, 'compou...",0.430,0.000,0.570,0.9390
1,1178162,4869189,2013-05-29,6452964,Charlotte,Great location for both airport and city - gre...,great location airport city great amenities h...,"{'neg': 0.0, 'neu': 0.5, 'pos': 0.5, 'compound...",0.500,0.000,0.500,0.9001
2,1178162,5003196,2013-06-06,6449554,Sebastian,We really enjoyed our stay at Islams house. Fr...,really enjoyed stay islams house outside house...,"{'neg': 0.093, 'neu': 0.623, 'pos': 0.284, 'co...",0.284,0.093,0.623,0.8443
3,1178162,5150351,2013-06-15,2215611,Marine,The room was nice and clean and so were the co...,room nice clean commodities close airport metr...,"{'neg': 0.0, 'neu': 0.5, 'pos': 0.5, 'compound...",0.500,0.000,0.500,0.9184
4,1178162,5171140,2013-06-16,6848427,Andrew,Great location. Just 5 mins walk from the Airp...,great location 5 mins walk airport station goo...,"{'neg': 0.0, 'neu': 0.508, 'pos': 0.492, 'comp...",0.492,0.000,0.508,0.8658
5,1178162,5198929,2013-06-17,6663826,Arndt,A truely exeptional place to stay. The hosts a...,truely exeptional place stay hosts amazingly f...,"{'neg': 0.0, 'neu': 0.725, 'pos': 0.275, 'comp...",0.275,0.000,0.725,0.8221
6,1178162,6702817,2013-08-21,8099222,Maurice,It was a really nice time in Boston - best pla...,really nice time boston best place stay bosto...,"{'neg': 0.0, 'neu': 0.471, 'pos': 0.529, 'comp...",0.529,0.000,0.471,0.9924
7,1178162,6873023,2013-08-28,7671888,Elodie,"Islam is a very nice guy ! Attentive, funny, h...",islam nice guy attentive funny gave recommand...,"{'neg': 0.0, 'neu': 0.589, 'pos': 0.411, 'comp...",0.411,0.000,0.589,0.9228
8,1178162,7646702,2013-09-28,8197342,Arkadiusz,"The place is really well furnished, pleasant a...",place really well furnished pleasant clean isl...,"{'neg': 0.064, 'neu': 0.401, 'pos': 0.536, 'co...",0.536,0.064,0.401,0.9836
9,1178162,8094418,2013-10-15,9040491,Matthew,Our stay at Islam's place was fantastic. We co...,stay islams place fantastic couldnt asked bett...,"{'neg': 0.089, 'neu': 0.563, 'pos': 0.348, 'co...",0.348,0.089,0.563,0.9307


In [49]:
## need to isolate English reviews

from nltk.sentiment.vader import SentimentIntensityAnalyzer
sid = SentimentIntensityAnalyzer()
from nltk.corpus import stopwords   # stopwords to detect language
from nltk import wordpunct_tokenize # function to split up our words

def get_language_likelihood(input_text):
    """Return a dictionary of languages and their likelihood of being the 
    natural language of the input text
    """
 
    input_text = str(input_text).lower()
    input_words = wordpunct_tokenize(input_text)
 
    language_likelihood = {}
    total_matches = 0
    for language in stopwords._fileids:
        language_likelihood[language] = len(set(input_words) &
                set(stopwords.words(language)))
 
    return language_likelihood
 
def get_language(input_text):
    """Return the most likely language of the given text
    """ 
    likelihoods = get_language_likelihood(input_text)
    return sorted(likelihoods, key=likelihoods.get, reverse=True)[0]

In [40]:
# test
get_language("Tomorrow we are going to eat a lot")

'english'

In [43]:
# test
get_language("Mañana vamos a cenar bastante pero ayer comí demasiado")

'spanish'

In [50]:
# get the language of each review

reviews['language'] = reviews['comments'].map(get_language)
reviews[:10]

,listing_id,id,date,reviewer_id,reviewer_name,comments,COMMENTS,sentiment,snt_pos,snt_neg,snt_neu,snt_compound,language
0,1178162,4724140,2013-05-21,4298113,Olivier,My stay at islam's place was really cool! Good...,stay islams place really cool good location 5m...,"{'neg': 0.0, 'neu': 0.57, 'pos': 0.43, 'compou...",0.430,0.000,0.570,0.9390,english
1,1178162,4869189,2013-05-29,6452964,Charlotte,Great location for both airport and city - gre...,great location airport city great amenities h...,"{'neg': 0.0, 'neu': 0.5, 'pos': 0.5, 'compound...",0.500,0.000,0.500,0.9001,english
2,1178162,5003196,2013-06-06,6449554,Sebastian,We really enjoyed our stay at Islams house. Fr...,really enjoyed stay islams house outside house...,"{'neg': 0.093, 'neu': 0.623, 'pos': 0.284, 'co...",0.284,0.093,0.623,0.8443,english
3,1178162,5150351,2013-06-15,2215611,Marine,The room was nice and clean and so were the co...,room nice clean commodities close airport metr...,"{'neg': 0.0, 'neu': 0.5, 'pos': 0.5, 'compound...",0.500,0.000,0.500,0.9184,english
4,1178162,5171140,2013-06-16,6848427,Andrew,Great location. Just 5 mins walk from the Airp...,great location 5 mins walk airport station goo...,"{'neg': 0.0, 'neu': 0.508, 'pos': 0.492, 'comp...",0.492,0.000,0.508,0.8658,english
5,1178162,5198929,2013-06-17,6663826,Arndt,A truely exeptional place to stay. The hosts a...,truely exeptional place stay hosts amazingly f...,"{'neg': 0.0, 'neu': 0.725, 'pos': 0.275, 'comp...",0.275,0.000,0.725,0.8221,english
6,1178162,6702817,2013-08-21,8099222,Maurice,It was a really nice time in Boston - best pla...,really nice time boston best place stay bosto...,"{'neg': 0.0, 'neu': 0.471, 'pos': 0.529, 'comp...",0.529,0.000,0.471,0.9924,english
7,1178162,6873023,2013-08-28,7671888,Elodie,"Islam is a very nice guy ! Attentive, funny, h...",islam nice guy attentive funny gave recommand...,"{'neg': 0.0, 'neu': 0.589, 'pos': 0.411, 'comp...",0.411,0.000,0.589,0.9228,english
8,1178162,7646702,2013-09-28,8197342,Arkadiusz,"The place is really well furnished, pleasant a...",place really well furnished pleasant clean isl...,"{'neg': 0.064, 'neu': 0.401, 'pos': 0.536, 'co...",0.536,0.064,0.401,0.9836,english
9,1178162,8094418,2013-10-15,9040491,Matthew,Our stay at Islam's place was fantastic. We co...,stay islams place fantastic couldnt asked bett...,"{'neg': 0.089, 'neu': 0.563, 'pos': 0.348, 'co...",0.348,0.089,0.563,0.9307,english


In [51]:
reviews['language'].value_counts()

english       64064
french         1285
danish         1173
spanish         630
german          535
dutch           213
italian         192
portuguese      101
russian          27
swedish          18
norwegian        17
romanian          7
finnish           6
turkish           4
hungarian         3
Name: language, dtype: int64

In [54]:
reviews_english = reviews[reviews['language'] == 'english']
reviews_english[:5]

,listing_id,id,date,reviewer_id,reviewer_name,comments,COMMENTS,sentiment,snt_pos,snt_neg,snt_neu,snt_compound,language
0,1178162,4724140,2013-05-21,4298113,Olivier,My stay at islam's place was really cool! Good...,stay islams place really cool good location 5m...,"{'neg': 0.0, 'neu': 0.57, 'pos': 0.43, 'compou...",0.430,0.000,0.570,0.9390,english
1,1178162,4869189,2013-05-29,6452964,Charlotte,Great location for both airport and city - gre...,great location airport city great amenities h...,"{'neg': 0.0, 'neu': 0.5, 'pos': 0.5, 'compound...",0.500,0.000,0.500,0.9001,english
2,1178162,5003196,2013-06-06,6449554,Sebastian,We really enjoyed our stay at Islams house. Fr...,really enjoyed stay islams house outside house...,"{'neg': 0.093, 'neu': 0.623, 'pos': 0.284, 'co...",0.284,0.093,0.623,0.8443,english
3,1178162,5150351,2013-06-15,2215611,Marine,The room was nice and clean and so were the co...,room nice clean commodities close airport metr...,"{'neg': 0.0, 'neu': 0.5, 'pos': 0.5, 'compound...",0.500,0.000,0.500,0.9184,english
4,1178162,5171140,2013-06-16,6848427,Andrew,Great location. Just 5 mins walk from the Airp...,great location 5 mins walk airport station goo...,"{'neg': 0.0, 'neu': 0.508, 'pos': 0.492, 'comp...",0.492,0.000,0.508,0.8658,english


In [69]:
num_neg_reviews = len(reviews_english[reviews_english['snt_compound'] < 0])
num_pos_reviews = len(reviews_english[reviews_english['snt_compound'] > 0])

print("number of negative reviews:", num_neg_reviews)
print("number of positive reviews:", num_pos_reviews)

print("percent negative reviews:", 100*(num_neg_reviews / num_pos_reviews))




number of negative reviews: 675
number of positive reviews: 62056
percent negative reviews: 1.0877272141291738


In [59]:
print("average:", average(reviews_english['snt_compound']))
print("median:", median(reviews_english['snt_compound']))

average: 0.85758607018
median: 0.9371
